# Prototype of VEM in M&M ASH model
This is the VEM part of M&M ASH model.

In [4]:
from dsc.utils import load_rds
import warnings, rpy2
warnings.filterwarnings("ignore") 
dat = load_rds('/home/gaow/Documents/GTExV8/Thyroid.Lung.FMO2.filled.rds')
import pandas as pd
dat['Y'] = pd.DataFrame(dat['Y'])

In [7]:
import numpy as np
from scipy.stats import multivariate_normal

## Data preview

In [4]:
dat['X']

matrix([[ 1.,  0.,  0., ...,  0.,  1.,  0.],
        [ 0.,  0.,  0., ...,  0.,  1.,  0.],
        [ 0.,  1.,  0., ...,  0.,  1.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  1.,  0.],
        [ 0.,  0.,  0., ...,  0.,  1.,  0.]])

In [6]:
dat['Y']

,Lung,Thyroid
0,0.770109,0.163481
1,0.777987,0.435890
2,-0.653612,-0.212373
3,0.006720,0.326649
4,-0.367918,-0.697831
5,-0.097376,0.840123
6,-0.003500,1.286902
7,-0.003500,0.115426
8,-0.300018,0.763787
9,-0.379263,-0.329763


## Initial values

We need to initialize M&M model with effect size estimates and a version of MASH results. We can use:

1. LD-convoluted effect estimate for mash model, to learn $\pi_t$ corresponding to $V_t$
2. Use multivariate LASSO to get the ordering of initialization of $X$, and initial $X$ values

### Multivariate LASSO 
For initialization of effects.
FIXME: not working yet.

In [ ]:
from sklearn import linear_model
import numpy as np
model = linear_model.lasso_path(dat['X'], dat['Y'], 0.1, fit_intercept = False)

Keyboard Interrupt


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.scatter(model.coef_[0], model.coef_[1], cmap="viridis")
ax = plt.gca()
plt.show()

In [ ]:
max(model.coef_[1])

### MASH priors and weights
FIXME: not working yet

## Core updates

In [17]:
# X is N by K matrix, X_norm is 1 by K vector of L2 norm of column k's
X_norm = np.linalg.norm(X, ord = 2, axis = 0)
X_std = X / X_norm
if test:
    np.testing.assert_array_almost_equal(np.sum(np.square(X_std), axis = 0), np.ones((1, X.shape[1])))
# B is M by K matrix
R_all = Y - X @ B
R = np.ones((X.shape[1], Y.shape[0], Y.shape[1])) * np.nan
E = np.ones((Y.shape[1], X.shape[1])) * np.nan
Rx = np.ones((Y.shape[1], X.shape[1])) * np.nan
wdE = np.ones((len(pi), X.shape[1])) * np.nan
# K is number of effects
for kk in range(X.shape[1]):
    # R[kk] is N x M, where M is number of conditions
    R[kk,:,:] = R_all + np.outer(X[:,kk], (B[kk,:].T))
    # E[kk] is M x 1
    E[:,kk] = (R[kk,:,:].T @ X_std[:,kk]).ravel()
    # Rx[kk] is M x 1
    Rx[:,kk] = (R[kk,:,:].T @ X[:,kk]).ravel()
    for tt in range(len(pi)):
        wdE[tt, kk] = multivariate_normal.pdf(E[kk], V[tt] + Lambda * -1) @ (1 / X_norm[kk])) * pi[tt]
    wdE_sum = sum(wdE[tt][kk])
    for tt in range(t):
        Sigma[tt][kk] = np.linalg.inv(np.diag(V[tt].shape[0]) + V[tt] @ X_norm[kk] @ Lambda) @ V[tt]
        mu[tt,,kk] = Sigma[tt][kk] @ Lambda @ Rx[kk]
        gamma[kk][tt] = wdE[tt][kk] / wdE_sum
    # dot product for weighted sums
    B[kk, ] = np.inner(mu[, , kk], gamma[kk])
# Recalculate h, a M by M matrix
H = np.diag(np.sum([np.sum([gamma[kk][tt] * ((mu[tt,,kk] - B[kk,]) ** 2 + np.diag(Sigma[tt][kk])) for tt in t]) for kk in k]))
# Update Lambda
Delta = np.diag(R_all.T @ R_all) + H
Lambda = X.shape[0] / np.diag(Delta)

SyntaxError: invalid syntax (<ipython-input-14-f31e21e5f360>, line 14)

In [15]:
X = dat['X']
Y = dat['Y']
B = np.zeros((X.shape[1], Y.shape[1]))
test = 1
kk  = 0
tt = 0
pi = np.array([0.2,0.8])
V = np.ones((2, Y.shape[1], Y.shape[1])) * np.nan
Lambda = np.diag(np.ones(Y.shape[1]))

matrix([[ 1.        ],
        [ 0.        ],
        [ 0.        ],
        [ 0.        ],
        [ 0.        ],
        [ 1.        ],
        [ 1.        ],
        [ 0.        ],
        [ 1.        ],
        [ 1.        ],
        [ 0.        ],
        [ 2.        ],
        [ 1.        ],
        [ 0.        ],
        [ 1.        ],
        [ 0.        ],
        [ 0.        ],
        [ 0.        ],
        [ 0.        ],
        [ 1.        ],
        [ 1.        ],
        [ 0.        ],
        [ 0.        ],
        [ 0.        ],
        [ 0.        ],
        [ 0.        ],
        [ 0.        ],
        [ 1.        ],
        [ 1.        ],
        [ 0.        ],
        [ 0.        ],
        [ 0.        ],
        [ 1.        ],
        [ 0.        ],
        [ 0.        ],
        [ 0.        ],
        [ 1.        ],
        [ 1.        ],
        [ 0.        ],
        [ 0.        ],
        [ 0.        ],
        [ 1.        ],
        [ 0.        ],
        [ 0